In [49]:
import os
import base64
import joblib
import pandas as pd
import numpy as np
import json
import joblib
import pickle
import math
import requests
from copy import deepcopy
import seaborn as sns
from uuid import uuid4
import json



In [50]:
# Assign the variable APP_NAME to the name of your heroku app
APP_NAME = 'railway-deployment-hack6-t3-production.up.railway.app'

# YOUR CODE HERE
# raise NotImplementedError()

In [51]:
# Test locally
# url = f"http://localhost:5000/predict"


# Testing the predict/update endpoint
url = "https://{}/predict".format(APP_NAME)


################################################
# Test with good payload
################################################

payload = {"admission_id": 82679, "patient_id": 81317898, "race": "Caucasian", "gender": "Male", "age": "Nan", "weight": "?", "admission_type_code": 3.0, "discharge_disposition_code": 3.0, "admission_source_code": 1, "time_in_hospital": 7, "payer_code": "MC", "medical_specialty": "Family/GeneralPractice", "has_prosthesis": False, "complete_vaccination_status": "Complete", "num_lab_procedures": 44.0, "num_procedures": 1, "num_medications": 13.0, "number_outpatient": 0, "number_emergency": 0, "number_inpatient": 2, "diag_1": "585", "diag_2": "491", "diag_3": "276", "number_diagnoses": 9, "blood_type": "O+", "hemoglobin_level": 15.6, "blood_transfusion": False, "max_glu_serum": "None", "A1Cresult": "None", "diuretics": "No", "insulin": "Yes", "change": "Ch", "diabetesMed": "Yes"}


r = requests.post(url, json=payload)
assert isinstance(r, requests.Response)
assert r.ok

response = r.json()

In [52]:
response

{'error': 'Observation ID: "82679" already exists', 'readmitted': 'Yes'}

In [53]:
# Test locally
# url = f"http://localhost:5000/predict"


# Testing the predict/update endpoint
url = "https://{}/update".format(APP_NAME)


################################################
# Test with good payload
################################################

payload = {"admission_id": 82679, "readmitted": "No"}

r = requests.post(url, json=payload)
assert isinstance(r, requests.Response)
assert r.ok

response = r.json()

In [54]:
response

{'actual_readmitted': 'No',
 'admission_id': 82679,
 'predicted_readmitted': 'Yes'}

In [1]:
import hashlib # for grading purposes
import math
import numpy as np
import pandas as pd
import requests
import sqlalchemy

In [2]:
username = 'postgres'
password = '238t5VKdRn4ZS1Vwt5KV'  # the password is not XXX by the way
host_name = 'containers-us-west-34.railway.app'
port = 7377
db_name = 'railway'

In [3]:
conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)
engine = sqlalchemy.create_engine(conn_str)

In [6]:
query = 'SELECT * FROM prediction;'


In [33]:
data_server = pd.read_sql_query(query, engine)

In [34]:
data_server

,id,admission_id,observation,readmitted,actual_readmitted
0,1,82679,"{'admission_id': 82679, 'patient_id': 81317898...",Yes,No
1,5,83311,"{'admission_id': 83311, 'patient_id': 18296960...",Yes,No
2,6,84417,"{'admission_id': 84417, 'patient_id': 62128908...",No,No
3,7,100530,"{'admission_id': 100530, 'patient_id': 2675520...",Yes,No
4,1223,86107,"{'admission_id': 86107, 'patient_id': 36549864...",No,No
...,...,...,...,...,...
1318,1325,84097,"{'admission_id': 84097, 'patient_id': 47351502...",No,No
1319,1327,96814,"{'admission_id': 96814, 'patient_id': 1733346,...",No,No
1320,1334,93935,"{'admission_id': 93935, 'patient_id': 32093892...",No,No
1321,1340,83908,"{'admission_id': 83908, 'patient_id': 18221502...",Yes,No


In [35]:
from sklearn.metrics import f1_score

In [36]:
data_server_clean = data_server.dropna()

In [37]:
data_server_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1323 non-null   int64 
 1   admission_id       1323 non-null   int64 
 2   observation        1323 non-null   object
 3   readmitted         1323 non-null   object
 4   actual_readmitted  1323 non-null   object
dtypes: int64(2), object(3)
memory usage: 51.8+ KB


In [38]:
f1_score(data_server_clean.actual_readmitted, data_server_clean.readmitted, pos_label='Yes')

0.22792937399678972

In [47]:
data_server_list = data_server.observation.values

In [51]:
import ast


In [61]:
data_server_values = pd.DataFrame([eval(data.replace("nan", "None" )) for data in data_server_list])

In [63]:
data_server_values

,admission_id,patient_id,race,gender,age,weight,admission_type_code,discharge_disposition_code,admission_source_code,time_in_hospital,...,number_diagnoses,blood_type,hemoglobin_level,blood_transfusion,max_glu_serum,A1Cresult,diuretics,insulin,change,diabetesMed
0,82679,81317898,Caucasian,Male,Nan,?,3.0,3.0,1,7,...,9.0,O+,15.6,False,None,None,No,Yes,Ch,Yes
1,83311,182969604,Caucasian,Female,[70-80),?,2.0,2.0,7,2,...,7.0,B-,14.8,False,None,None,No,Yes,Ch,Yes
2,84417,62128908,Caucasian,Female,[70-80),?,5.0,6.0,1,3,...,9.0,O+,12.9,False,None,Norm,No,No,No,Yes
3,100530,26755200,Afro American,Male,[50-60),?,2.0,18.0,2,6,...,9.0,O+,14.1,False,None,None,No,No,No,Yes
4,86107,36549864,Caucasian,Male,[50-60),?,2.0,1.0,1,2,...,5.0,A+,14.3,False,None,None,No,Yes,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,84097,47351502,Caucasian,Male,[80-90),?,1.0,23.0,7,12,...,9.0,A-,14.2,False,None,None,No,No,No,Yes
1319,96814,1733346,AfricanAmerican,Female,[60-70),?,1.0,1.0,7,1,...,4.0,AB+,13.4,False,None,Norm,No,Yes,No,Yes
1320,93935,32093892,Caucasian,Male,[70-80),?,1.0,1.0,7,6,...,8.0,O+,15.4,True,NONE,None,No,No,No,No
1321,83908,182215026,Hispanic,Female,[60-70),?,NaN,1.0,7,6,...,4.0,O+,13.3,False,None,Norm,No,No,No,Yes
